In [1]:
# ! pip install --upgrade --quiet langchain langchain-community langchainhub langchain-openai openinference-instrumentation-langchain langchain-text-splitters faiss-cpu 'arize-phoenix[evals]' 


In [2]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [5]:
import phoenix as px
session = px.launch_app()

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [6]:
from openinference.instrumentation.langchain import LangChainInstrumentor
LangChainInstrumentor().instrument()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader("../city_data", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
docs = loader.load_and_split(text_splitter=text_splitter)

Error loading file ../city_data/Chicago.txt


ImportError: unstructured package not found, please install it with `pip install unstructured`

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(
    docs, 
    embeddings)

In [7]:
retriever = db.as_retriever()

In [8]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "search_cities",
    "Searches and returns excerpts from Wikipedia entries of many cities.",
)
tools = [tool]

In [9]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, verbose=True)

In [12]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [23]:
result = agent_executor.invoke(
    {
        "input": "Which city has the best food scene out of Lisbon, Houston, Berlin, Moscow, and Paris?"
    }
)

In [24]:
result["output"]

"Based on the information retrieved, here is a brief overview of the food scenes in each city:\n\n1. **Lisbon**: Lisbon is known for its vibrant food scene with a variety of culinary experiences. The city hosts events like the Lisbon Gay & Lesbian Film Festival, Lisboarte, and the Lisbon Fish and Flavours festival. Lisbon also has fine examples of Portuguese azulejo tile art and is recognized as an alpha-level global city with a significant presence in finance, commerce, fashion, and tourism.\n\n2. **Houston**: Houston has a diverse food scene with a mix of local and international cuisines. The city is known for its music scene, particularly in hip hop, and has become a destination for many college-educated individuals. Houston is also home to the National Museum of Funeral History and hosts various musical acts in different genres.\n\n3. **Berlin**: Berlin's food scene is diverse and vibrant, with a rich history of culinary influences. The city is known for its nightlife, with a varie